In [390]:
import ast
import numpy as np
import pandas as pd

from collections import defaultdict

In [426]:
df = pd.read_csv('data/EPIC_100_annotated_episodes_fixed_timestamps.csv', header=0)

In [427]:
df

,Unnamed: 0,narration_id,participant_id,video_id,narration_timestamp,start_timestamp,stop_timestamp,start_frame,stop_frame,narration,...,all_nouns,all_noun_classes,grid_location,prior_grid,lhand_vposition,lhand_hposition,rhand_vposition,rhand_hposition,hand_in_use,Unnamed: 22
0,318,P01_105_0,P01,P01_105,00:01.1,00:01.0,00:02.2,51,109,pick up container,...,['container'],[21],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,319,P01_105_1,P01,P01_105,00:02.6,00:02.3,00:03.5,114,175,open cupboard,...,['cupboard'],[3],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,430,P01_105_2,P01,P01_105,00:04.6,00:03.8,00:05.0,190,248,pick up container,...,['container'],[21],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,541,P01_105_3,P01,P01_105,00:05.1,00:05.2,00:05.7,258,283,put down containers,...,['container'],[21],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,652,P01_105_4,P01,P01_105,00:06.4,00:05.8,00:07.2,288,358,open container,...,['container'],[21],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23554,38582,P37_102_145,P37,P37_102,11:12.4,11:06.7,11:13.0,33334,33651,take plate,...,['plate'],[2],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23555,38583,P37_102_146,P37,P37_102,11:14.2,11:13.0,11:15.4,33651,33771,put plates down,...,['plate'],[2],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23556,38584,P37_102_147,P37,P37_102,11:16.2,11:15.4,11:16.8,33772,33840,take packing bag,...,['bag:packing'],[19],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23557,38585,P37_102_148,P37,P37_102,11:17.2,11:16.8,11:17.7,33842,33882,put packing bag down,...,['bag:packing'],[19],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### get info from annotation df

In [353]:
# define dict for sequences {id: sequence, ...}
# get first instance of take [item] if item not already in sequence
# starting positions: get grid loc/hand pos before taking action
# take [item] = position [item]?

# for start positions: if previous row is annotated: take z position from there

In [428]:
sequences = defaultdict(list)
start_positions = defaultdict(list)
item_positions = defaultdict(list)
action_list = defaultdict(list)

In [429]:
for row in range(0,len(df)):
    #print(row)
    if'take' in df.at[row,'narration']\
        or 'pick up' in df.at[row,'narration']\
        or 'grab' in df.at[row,'narration']\
        or 'get' in df.at[row,'narration']\
        or 'taking' in df.at[row,'narration']:
        if df.at[row,'noun'] not in sequences[df.at[row,'video_id']]:
            sequences[df.at[row,'video_id']].append(df.at[row,'noun'])
            previous_position = [df.at[row, 'prior_grid']]
            
            if not row == 0 and not pd.isna(df.at[int(row)-1, 'hand_in_use']):
                if df.at[row - 1, 'hand_in_use'] == 'right':
                    previous_z = df.at[row - 1, 'rhand_vposition']
                else:
                    previous_z = df.at[row - 1, 'lhand_vposition']
            else:
                previous_z = np.nan
            
            if df.at[row, 'hand_in_use'] == 'right':
                current_z = df.at[row,'rhand_vposition']
            else:
                current_z = df.at[row,'lhand_vposition']
            previous_position.append(previous_z)
            
            start_positions[df.at[row,'video_id']].append(previous_position)
            item_position = [df.at[row, 'grid_location']]
            item_position.append(current_z)
            
            item_positions[df.at[row, 'video_id']].append(item_position)
            action_list[df.at[row,'video_id']].append(df.at[row,'narration'])

In [430]:
sequences

defaultdict(list,
            {'P01_105': ['container',
              'lid',
              'pot',
              'tray',
              'fork',
              'cup',
              'board:cutting',
              'mushroom',
              'choi:pak',
              'tofu',
              'carrot',
              'courgette',
              'knife',
              'wok',
              'pan',
              'dice:courgette',
              'slice:courgette',
              'ic:courgette',
              'peeler',
              'oil',
              'salt',
              'skin:carrot',
              'spatula',
              'onion',
              'card',
              'bag',
              'box',
              'cloth',
              'pak',
              'colander',
              'bowl',
              'leaf:pak:choi',
              'bit:pak:choi',
              'garlic',
              'bit:food',
              'noodle',
              'saucepan',
              'scissors',
              'kettle',
          

In [431]:
start_positions

defaultdict(list,
            {'P01_105': [[nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              ['(3,1)', nan],
              ['(1,1)', nan],
              ['(0,0)', nan],
              ['(0,0)', nan],
              ['(0,0)', nan],
              ['(0,0)', nan],
              ['(0,0)', nan],
              ['(1,1)', nan],
              [nan, '1'],
              [nan, nan],
              [nan, nan],
              ['(2,1)', '0'],
              [nan, nan],
              ['(2,1)', nan],
              [nan, '0'],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, '0'],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, n

In [432]:
action_list

defaultdict(list,
            {'P01_105': ['pick up container',
              'pick up lid',
              'pick up pot',
              'pick up tray',
              'pick up fork and cup',
              'pick up cup',
              'pick up cutting board',
              'pick up mushrooms',
              'pick up pak choi',
              'pick up tofu',
              'pick up carrot',
              'pick up courgette',
              'take knife',
              'take wok',
              'take pans',
              'pick up courgette dice',
              'move courgette slices',
              'pick up courgette dice',
              'pick up peeler',
              'pick up oil',
              'pick up salt',
              'take carrot skin',
              'pick up spatula',
              'pick up onion',
              'take card off box',
              'pick up bag',
              'pick up box',
              'pick up cloth',
              'pick up pak choi end',
              'pick up co

In [396]:
item_positions

defaultdict(list,
            {'P01_105': [[nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              ['(1,1)', '1'],
              ['(0,0)', '0'],
              ['(0,0)', '0'],
              ['(0,0)', '0'],
              ['(0,0)', '0'],
              ['(0,0)', '0'],
              ['(3,1)', '7'],
              ['(1,1)', '1'],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              ['(2,1)', '0'],
              [nan, nan],
              ['(1,1)', '7'],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, nan],
              [nan, n

### fix start/item positions -> add z coordinate

In [433]:
z_coordinates = {0:2, 1:1, 2:0, 3:1, 4:2, 5:3, 6:4, 7:3}

In [434]:
start_positions2 = {}

for key, values in start_positions.items():
    start_positions2[key] = {}
    for item in range(0, len(values)):
        item_name = sequences[key][item]
        
        if not pd.isna(values[item][0]):
            new_item = ast.literal_eval(values[item][0])
            new_item = [x for x in new_item]
            
            if not pd.isna(values[item][1]):
                #print(values[item][1])
                new_item.append(z_coordinates[int(values[item][1])])
                #print(z_coordinates[int(values[item][1])], '\n_____')
            else:
                new_item.append(2)
            start_positions2[key][item_name] = new_item
            
        else:
            start_positions2[key][item_name] = []

In [435]:
item_positions2 = {}

for key, values in item_positions.items():
    item_positions2[key] = {}
    for item in range(0, len(values)):
        item_name = sequences[key][item]

        if not pd.isna(values[item][0]):
            new_item = ast.literal_eval(values[item][0])
            new_item = [x for x in new_item]
            new_item.append(z_coordinates[int(values[item][1])])
            item_positions2[key][item_name] = new_item
            
        else:
            item_positions2[key][item_name] = []

### create new df

In [436]:
task_envs = pd.DataFrame()

In [437]:
task_envs['ID'] = df['video_id'].unique()

In [438]:
task_envs['sequence_long'] = 'x'
task_envs['start_positions'] = 'x'
task_envs['coordinates'] = 'x'
task_envs = episode_df.astype(object)

In [439]:
for row in range(0, len(task_envs)):
    ID = task_envs.at[row, 'ID']
    #print(ID)
    sequence = sequences[ID]
    start_pos = start_positions2[ID]
    coords = item_positions2[ID]
    task_envs.at[row, 'sequence_long'] = sequence
    task_envs.at[row, 'start_positions'] = start_pos
    task_envs.at[row, 'coordinates'] = coords

### filter out too short sequences

In [440]:
task_envs = task_envs[task_envs['sequence_long'].map(len) > 2]

In [441]:
task_envs

,ID,sequence_long,start_positions,coordinates
0,P01_105,"[container, lid, pot, tray, fork, cup, board:c...","{'container': [], 'lid': [], 'pot': [], 'tray'...","{'container': [], 'lid': [], 'pot': [], 'tray'..."
1,P01_109,"[rucksack, egg, onion, foil:aluminium, potato,...","{'rucksack': [], 'egg': [], 'onion': [], 'foil...","{'rucksack': [], 'egg': [], 'onion': [], 'foil..."
2,P02_101,"[milk, bread, toast, plate, knife, butter, jam...","{'milk': [], 'bread': [], 'toast': [], 'plate'...","{'milk': [], 'bread': [], 'toast': [], 'plate'..."
3,P02_104,"[tupperware, pasta, pan, kettle, glove, strain...","{'tupperware': [], 'pasta': [], 'pan': [], 'ke...","{'tupperware': [], 'pasta': [], 'pan': [], 'ke..."
4,P02_109,"[board:chopping, onion, knife, piece:onion, pa...","{'board:chopping': [], 'onion': [], 'knife': [...","{'board:chopping': [], 'onion': [], 'knife': [..."
...,...,...,...,...
104,P35_103,"[mixer, pot, spoon, salt, cloth, container, pa...","{'mixer': [], 'pot': [], 'spoon': [], 'salt': ...","{'mixer': [], 'pot': [], 'spoon': [], 'salt': ..."
105,P35_104,"[food, sponge, plate, container, glass, cloth,...","{'food': [], 'sponge': [], 'plate': [], 'conta...","{'food': [], 'sponge': [], 'plate': [], 'conta..."
106,P35_105,"[saucepan, cloth, container, mushroom, clip, f...","{'saucepan': [], 'cloth': [], 'container': [],...","{'saucepan': [], 'cloth': [], 'container': [],..."
107,P37_101,"[box:plastic, stool, pot:cooking, pot, tray:pl...","{'box:plastic': [], 'stool': [], 'pot:cooking'...","{'box:plastic': [], 'stool': [], 'pot:cooking'..."


In [442]:
task_envs.to_csv('epic_100_task_envs.csv', header=True, index=False)